In [16]:
num_base_samp_points=3
num_additional_samp_points=2

interp="""(((
    source_arr[Int(floor( shared_arr[threadIdx().x,1])), Int(floor( shared_arr[threadIdx().x,2])), Int(floor( shared_arr[threadIdx().x,3]))] * (1 - (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1])))) +
    source_arr[Int(ceil( shared_arr[threadIdx().x,1])), Int(floor( shared_arr[threadIdx().x,2])), Int(floor( shared_arr[threadIdx().x,3]))] * (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1])))
    )
      *
      (1 - (shared_arr[threadIdx().x,2] - Int(floor( shared_arr[threadIdx().x,2])))) +
      (source_arr[Int(floor( shared_arr[threadIdx().x,1])), Int(ceil( shared_arr[threadIdx().x,2])), Int(floor( shared_arr[threadIdx().x,3]))] * (1 - (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1]))))
       +
       source_arr[Int(ceil( shared_arr[threadIdx().x,1])), Int(ceil( shared_arr[threadIdx().x,2])), Int(floor( shared_arr[threadIdx().x,3]))] * (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1]))))
      *
      (shared_arr[threadIdx().x,2] - Int(floor( shared_arr[threadIdx().x,2]))))
     *
     (1 - (shared_arr[threadIdx().x,3] - Int(floor( shared_arr[threadIdx().x,3]))))
     +
     ((source_arr[Int(floor( shared_arr[threadIdx().x,1])), Int(floor( shared_arr[threadIdx().x,2])), Int(ceil( shared_arr[threadIdx().x,3]))] * (1 - (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1]))))
       +
       source_arr[Int(ceil( shared_arr[threadIdx().x,1])), Int(floor( shared_arr[threadIdx().x,2])), Int(ceil( shared_arr[threadIdx().x,3]))] * (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1]))))
      *
      (1 - (shared_arr[threadIdx().x,2] - Int(floor( shared_arr[threadIdx().x,2])))) +
      (source_arr[Int(floor( shared_arr[threadIdx().x,1])), Int(ceil( shared_arr[threadIdx().x,2])), Int(ceil( shared_arr[threadIdx().x,3]))] * (1 - (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1]))))
       +
       source_arr[Int(ceil( shared_arr[threadIdx().x,1])), Int(ceil( shared_arr[threadIdx().x,2])), Int(ceil( shared_arr[threadIdx().x,3]))] * (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1]))))
      *
      (shared_arr[threadIdx().x,2] - Int(floor( shared_arr[threadIdx().x,2]))))
     *
     (shared_arr[threadIdx().x,3] - Int(floor( shared_arr[threadIdx().x,3]))))

"""



res=""
for point_num in 1:num_base_samp_points
res=res*"""
#we get the diffrence between the sv center and the triangle center
      shared_arr[threadIdx().x,1]= ((tetr_dat[index,5,1]-tetr_dat[index,1,1])/($num_base_samp_points+1))
      shared_arr[threadIdx().x,2]=((tetr_dat[index,5,2]-tetr_dat[index,1,2])/($num_base_samp_points+1))
      shared_arr[threadIdx().x,3]=((tetr_dat[index,5,3]-tetr_dat[index,1,3])/($num_base_samp_points+1))



      ##calculate weight of the point
      #first distance from next and previous point on the line between sv center and triangle center
      shared_arr[threadIdx().x,4]=sqrt((shared_arr[threadIdx().x,1])^2 +(shared_arr[threadIdx().x,2])^2+(shared_arr[threadIdx().x,3])^2)*2 #distance between main sample points (two times for distance to previous and next)
      #now we get the distance to the lines that get from sv center to the triangle corners - for simplicity
      # we can assume that sv center location is 0.0,0.0,0.0 as we need only diffrences 
      #now we get the location of sample point

      shared_arr[threadIdx().x,1]= tetr_dat[index,1,1]+(shared_arr[threadIdx().x,1]*$point_num)
      shared_arr[threadIdx().x,2]= tetr_dat[index,1,2]+(shared_arr[threadIdx().x,2]*$point_num)
      shared_arr[threadIdx().x,3]= tetr_dat[index,1,3]+(shared_arr[threadIdx().x,3]*$point_num)

"""
    for triangle_corner_num in 1:3
        if(point_num==num_base_samp_points)
            res=res*"""

            shared_arr[threadIdx().x,4]+=sqrt( ((tetr_dat[index,$triangle_corner_num+1,1]-shared_arr[threadIdx().x,1])
                  *(1/(num_additional_samp_points+1)))^2
                  +((tetr_dat[index,$triangle_corner_num+1,2]-shared_arr[threadIdx().x,2])
                  *(1/(num_additional_samp_points+1)))^2
                  +((tetr_dat[index,$triangle_corner_num+1,3]-shared_arr[threadIdx().x,3])
                  *(1/(num_additional_samp_points+1)))^2)

                   """
        else
            res=res*"""

            shared_arr[threadIdx().x,4]+=sqrt(((((tetr_dat[index,1,2]-tetr_dat[index,$triangle_corner_num+1,2])*(tetr_dat[index,1,3]-shared_arr[threadIdx().x,3]) - (tetr_dat[index,1,3]-tetr_dat[index,$triangle_corner_num+1,3])*(tetr_dat[index,1,2]-shared_arr[threadIdx().x,2]))^2)+
              (((tetr_dat[index,1,3]-tetr_dat[index,$triangle_corner_num+1,3])*(tetr_dat[index,1,1]-shared_arr[threadIdx().x,1]) - (tetr_dat[index,1,1]-tetr_dat[index,$triangle_corner_num+1,1])*(tetr_dat[index,1,3]-shared_arr[threadIdx().x,3]))^2)+
              ((tetr_dat[index,1,1]-tetr_dat[index,$triangle_corner_num+1,1])*(tetr_dat[index,1,2]-shared_arr[threadIdx().x,2]) - (tetr_dat[index,1,2]-tetr_dat[index,$triangle_corner_num+1,2])*(tetr_dat[index,1,1]-shared_arr[threadIdx().x,1]))^2)) / sqrt((tetr_dat[index,$triangle_corner_num+1,2]-tetr_dat[index,1,2])^2+(tetr_dat[index,$triangle_corner_num+1,3]-tetr_dat[index,1,3])^2+(tetr_dat[index,$triangle_corner_num+1,1]-tetr_dat[index,1,1])^2)
              out_sampled_points[index,$point_num,1]=$interp(shared_arr,source_arr)
              #saving sample points coordinates mainly for debugging and visualizations
              out_sampled_points[index,$point_num,3]=shared_arr[threadIdx().x,1]
              out_sampled_points[index,$point_num,4]=shared_arr[threadIdx().x,2]
              out_sampled_points[index,$point_num,5]=shared_arr[threadIdx().x,3]

            """            
        end    

    end#triangle_corner_num
    res=res*""" out_sampled_points[index,$point_num,2]= (((shared_arr[threadIdx().x,4])/5)^3)
    out_sampled_points[index,$point_num,1]=$interp(shared_arr,source_arr)
    #saving sample points coordinates mainly for debugging and visualizations
    out_sampled_points[index,$point_num,3]=shared_arr[threadIdx().x,1]
    out_sampled_points[index,$point_num,4]=shared_arr[threadIdx().x,2]
    out_sampled_points[index,$point_num,5]=shared_arr[threadIdx().x,3]
    
    
    """

end


for n_add_samp in UInt8(1):UInt8(num_additional_samp_points)
    for triangle_corner_num in UInt8(1):UInt8(3)
        res=res*""" 
        shared_arr[threadIdx().x,1]=((tetr_dat[index,$triangle_corner_num+1,1]-out_sampled_points[index,num_base_samp_points,3])/($num_additional_samp_points+1))
        shared_arr[threadIdx().x,2]=((tetr_dat[index,$triangle_corner_num+1,2]-out_sampled_points[index,num_base_samp_points,4])/($num_additional_samp_points+1))
        shared_arr[threadIdx().x,3]=((tetr_dat[index,$triangle_corner_num+1,3]-out_sampled_points[index,num_base_samp_points,5])/($num_additional_samp_points+1))

        #### calculate weight of the point 
        #first we get distance to the previous and next point on the line between the last main sample point and the triangle corner
        shared_arr[threadIdx().x,4]=sqrt( shared_arr[threadIdx().x,1]^2+shared_arr[threadIdx().x,2]^2+shared_arr[threadIdx().x,3]^2)*2
        #now we get the location of sample point
        shared_arr[threadIdx().x,1]= out_sampled_points[index,num_base_samp_points,3]+(shared_arr[threadIdx().x,1]*($n_add_samp))
        shared_arr[threadIdx().x,2]= out_sampled_points[index,num_base_samp_points,4]+(shared_arr[threadIdx().x,2]*($n_add_samp))
        shared_arr[threadIdx().x,3]= out_sampled_points[index,num_base_samp_points,5]+(shared_arr[threadIdx().x,3]*($n_add_samp))

        """
        for tetr_dat_index in UInt8(1):UInt8(4)
            if(tetr_dat_index!=(triangle_corner_num+1) )
                res=res*""" 

                shared_arr[threadIdx().x,4]+=sqrt(((((tetr_dat[index,$triangle_corner_num+1,2]-tetr_dat[index,$tetr_dat_index,2])*(tetr_dat[index,$triangle_corner_num+1,3]-shared_arr[threadIdx().x,3]) - (tetr_dat[index,$triangle_corner_num+1,3]-tetr_dat[index,$tetr_dat_index,3])*(tetr_dat[index,$triangle_corner_num+1,2]-shared_arr[threadIdx().x,2]))^2)+
                (((tetr_dat[index,$triangle_corner_num+1,3]-tetr_dat[index,$tetr_dat_index,3])*(tetr_dat[index,$triangle_corner_num+1,1]-shared_arr[threadIdx().x,1]) - (tetr_dat[index,$triangle_corner_num+1,1]-tetr_dat[index,$tetr_dat_index,1])*(tetr_dat[index,$triangle_corner_num+1,3]-shared_arr[threadIdx().x,3]))^2)+
                ((tetr_dat[index,$triangle_corner_num+1,1]-tetr_dat[index,$tetr_dat_index,1])*(tetr_dat[index,$triangle_corner_num+1,2]-shared_arr[threadIdx().x,2]) - (tetr_dat[index,$triangle_corner_num+1,2]-tetr_dat[index,$tetr_dat_index,2])*(tetr_dat[index,$triangle_corner_num+1,1]-shared_arr[threadIdx().x,1]))^2)) / sqrt((tetr_dat[index,$tetr_dat_index,2]-tetr_dat[index,$triangle_corner_num+1,2])^2+(tetr_dat[index,$tetr_dat_index,3]-tetr_dat[index,$triangle_corner_num+1,3])^2+(tetr_dat[index,$tetr_dat_index,1]-tetr_dat[index,$triangle_corner_num+1,1])^2)

                
                """
            end#if
        end#tetr_dat_index    
        res=res*""" 
        out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,2]=(((shared_arr[threadIdx().x,4])/5)^3)      
        #performing interpolation result is in var2 and it get data from shared_arr
        #saving the result of interpolated value to the out_sampled_points
        out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,1]=$interp(shared_arr,source_arr)
        # #saving sample points coordinates
        out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,3]=shared_arr[threadIdx().x,1]
        out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,4]=shared_arr[threadIdx().x,2]
        out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,5]=shared_arr[threadIdx().x,3]
        
        """


    end#triangle_corner_num
    
end#n_add_samp    
# Specify the file path
filepath = "/home/jm/projects_new/superVoxelJuliaCode/superVoxelJuliaCode/src/old/unrolled.txt"

# Open the file in write mode
file = open(filepath, "w")

# Write the value of res to the file
write(file, res)

# Close the file
close(file)


In [17]:
a=" r "
b= " u "
a*b

" r  u "

In [3]:
using KernelAbstractions


# N = @uniform @groupsize()[1]
# M = @uniform @groupsize()[2]

# +1 to avoid bank conflicts on shared memory
tile = @localmem eltype(output) (N+BANK, M)

@kernel function mul2_kernel(A)
  @groupsize()
    I = @index(Global)
    A[I] = 2 * A[I]
  end
tile = @localmem eltype(output) (N+BANK, M) 
dev = CPU()
A = ones(1024, 1024)
ev = mul2_kernel(dev, 32)(A, ndrange=size(A))
synchronize(dev)
all(A .== 2.0)


ERROR: LoadError: ArgumentError: Package PrecompileTools [aea7be01-6a6a-4083-8856-8a6e6704d82a] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.

Stacktrace:
  [1] _require(pkg::Base.PkgId, env::String)
    @ Base ./loading.jl:1926
  [2] __require_prelocked(uuidkey::Base.PkgId, env::String)
    @ Base ./loading.jl:1812
  [3] #invoke_in_world#3
    @ ./essentials.jl:926 [inlined]
  [4] invoke_in_world
    @ ./essentials.jl:923 [inlined]
  [5] _require_prelocked(uuidkey::Base.PkgId, env::String)
    @ Base ./loading.jl:1803
  [6] macro expansion
    @ ./loading.jl:1790 [inlined]
  [7] macro expansion
    @ ./lock.jl:267 [inlined]
  [8] __require(into::Module, mod::Symbol)
    @ Base ./loading.jl:1753
  [9] #invoke_in_world#3
    @ ./essentials.jl:926 [inlined]
 [10] invoke_in_world
    @ ./essentials.jl:923 [inlined]
 [11] require(into::Module, mod::Symbol)
    @ Base ./loading.jl:1746
 [12] include
    @ ./Base.jl:495 [inlin

ErrorException: Failed to precompile KernelAbstractions [63c18a36-062a-441e-b654-da1e3ab1ce7c] to "/home/jm/.julia/compiled/v1.10/KernelAbstractions/jl_ejq6G6".